IMPORT PACKAGES AND DATA

In [2]:
import pandas as pd
import numpy as np


In [3]:
all_bid = pd.read_pickle(r"Data\all_bids_hackathon_masked.pkl")
train_df=pd.read_csv(r"Data\all_events_data_train.csv")
rules_df=pd.read_csv(r"Data\auction_rules_masked.csv")
fx_conversion=pd.read_csv(r"Data\fx_conversion_rate.csv")
submission_df=pd.read_csv(r"Data\hackathon_submission_event_details_masked.csv")
iv=pd.read_pickle(r"Data\participant_initial_value_masked.pkl")
invit_df=pd.read_pickle(r"Data\participant_invitation_masked.pkl")

Preprocess

In [4]:
train_df['data']='train'
submission_df['data']='submission_df'


In [5]:
full_df = pd.concat([train_df,submission_df])

In [6]:
full_df.reset_index(drop=True,inplace=True)

In [7]:
# df = pd.merge(train_df,rules_df,how='left',on='event_id')
df = pd.merge(full_df,rules_df,how='left',on='event_id')

In [8]:
all_bid_lot = pd.merge(all_bid,train_df[['event_id','Categ_L1']],on="event_id",how="left")

In [9]:
df['Efficiency']=(df['bestbid_usd']-df['finalspend_usd'])/df['bestbid_usd']

In [10]:
df['bidding_date'] = pd.to_datetime(df['bidding_date'])

# Determine if the date is a weekend (Saturday=5, Sunday=6) or weekday (Mon-Fri=0-4)
df['is_weekend'] = df['bidding_date'].dt.weekday >= 5  # Saturday and Sunday are >= 5

# Alternatively, to get 'Weekday' or 'Weekend' as a label:
df['day_type'] = df['bidding_date'].dt.weekday.apply(lambda x: 'Weekend' if x >= 5 else 'Weekday')

In [11]:
participants = invit_df.groupby(['event_id','invitation_status'])['participant'].count().reset_index()
participants['invitation_status'].unique()
participants = participants[participants['invitation_status'].isin(['Invited','Invited-Incumbent'])]
participants  =participants.groupby('event_id')['participant'].sum().reset_index()

In [12]:
item_df = invit_df.groupby(['event_id','invitation_status'])['item_name'].nunique().reset_index()
item_df['invitation_status'].unique()
item_df = item_df[item_df['invitation_status'].isin(['Invited','Invited-Incumbent'])]
item_df  =item_df.groupby('event_id')['item_name'].sum().reset_index()

In [13]:
df=pd.merge(df,participants,"left",on='event_id')
df=pd.merge(df,item_df,"left",on='event_id')
df.columns = df.columns.str.replace(' ', '')
df.columns = df.columns.str.lower()

In [14]:
df['ratio']=df['participant']/df['item_name']

In [15]:
float_columns = df.select_dtypes(include=['float']).columns
float_columns

Index(['baselinespend_usd', 'bestbid_usd', 'finalspend_usd',
       'effectiveness_usd', 'bidrankthattriggersovertime',
       'overtimeperiod(minutes)', 'startovertimeifbidsubmittedwithin(minutes)',
       'efficiency', 'participant', 'item_name', 'ratio'],
      dtype='object')

In [16]:
numerical_df = df[float_columns]

In [17]:
#time
categorical_columns = df.select_dtypes(include=['object', 'category']).columns


In [18]:
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)
df_encoded.drop('bidding_date',axis=1,inplace=True)

In [19]:

df_encoded = df_encoded.replace([np.nan, np.inf, -np.inf], 0)
df_encoded = df_encoded.astype(int)
df_filtered = df_encoded.loc[:, ~df_encoded.columns.str.contains('time', case=False)]
df_filtered = df_filtered.loc[:, ~df_filtered.columns.str.contains('date', case=False)]
df_filtered = df_filtered.loc[:, ~df_filtered.columns.str.contains('currency', case=False)]
df_filtered = df_filtered.loc[:, ~df_filtered.columns.str.contains('event_id_', case=False)]
df_filtered = df_filtered.loc[:, ~df_filtered.columns.str.contains('username', case=False)]
df_filtered = df_filtered.loc[:, ~df_filtered.columns.str.contains('commodity_proms', case=False)]
df_filtered = df_filtered.loc[:, ~df_filtered.columns.str.contains('efficiency', case=False)]
df_filtered = df_filtered.loc[:, ~df_filtered.columns.str.contains('biddingperiod', case=False)]

In [20]:
time_df = df.loc[:, df.columns.str.contains('time|period', case=False)]
time_df.drop('allowbiddingovertime',axis=1,inplace=True)
datetimedf=time_df[['overtimeperiod(minutes)','starttime']]

time_cols_list = ['reviewtimeperiod','runningtimeforthefirstlot','timebetweenlotclosing','biddingperiod']
for col in time_cols_list:
    print(col)
    time_cols = time_df[[col]]
    time_cols = time_cols.replace("nan", " ")
    time_cols[['value', 'unit']] = time_cols[col].str.split(' ', expand=True)
 
    time_cols['unit'] = time_cols["unit"].replace(
        {
            "" : 0,
            "Minutes" : 1,
            "Days": 60*24,
            "Hours" : 60,
            "Day" : 60*24,
            "Hour" : 60
        }
    )
    time_cols['unit'] = time_cols["unit"].fillna(0)
    time_cols['value'] = time_cols["value"].fillna("0")
    time_cols['value'] = time_cols['value'].replace("", "0")
    time_cols['value'] = time_cols['value'].astype(int)
    time_cols[col] = time_cols['unit'] * time_cols['value']
 
    time_df[col] = time_cols[col]

C:\Users\40107607\AppData\Local\Temp\ipykernel_2764\1483730146.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_df.drop('allowbiddingovertime',axis=1,inplace=True)
C:\Users\40107607\AppData\Local\Temp\ipykernel_2764\1483730146.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_df[col] = time_cols[col]


reviewtimeperiod
runningtimeforthefirstlot
timebetweenlotclosing
biddingperiod


C:\Users\40107607\AppData\Local\Temp\ipykernel_2764\1483730146.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_df[col] = time_cols[col]
C:\Users\40107607\AppData\Local\Temp\ipykernel_2764\1483730146.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_df[col] = time_cols[col]
C:\Users\40107607\AppData\Local\Temp\ipykernel_2764\1483730146.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [21]:
time_df = time_df[['bidrankthattriggersovertime','biddingperiod','overtimeperiod(minutes)','reviewtimeperiod','timebetweenlotclosing']]

In [22]:
time_df = time_df.fillna(0)

In [23]:
time_df = pd.concat([time_df,datetimedf],axis=1)

In [24]:
numerical_df.rename(columns={'item_name':'itemno'},inplace=True)


C:\Users\40107607\AppData\Local\Temp\ipykernel_2764\2140680625.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerical_df.rename(columns={'item_name':'itemno'},inplace=True)


In [25]:
numerical_df.drop(['bidrankthattriggersovertime','overtimeperiod(minutes)','startovertimeifbidsubmittedwithin(minutes)'],axis=1,inplace=True)

C:\Users\40107607\AppData\Local\Temp\ipykernel_2764\444128718.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numerical_df.drop(['bidrankthattriggersovertime','overtimeperiod(minutes)','startovertimeifbidsubmittedwithin(minutes)'],axis=1,inplace=True)


In [26]:
df_final=pd.concat([numerical_df,time_df,df_filtered],axis=1)

In [27]:
df_final= df_final.fillna(0)

In [28]:
full_df.columns.to_list()

['zone',
 'bidding_date',
 'event_format',
 'Currency_PROms',
 'Commodity_PROms',
 'baselinespend_usd',
 'bestbid_usd',
 'finalspend_usd',
 'effectiveness_usd',
 'event_template',
 'dag_categ',
 'Categ_L1',
 'Categ_L2',
 'event_id',
 'username',
 'data']

In [29]:
result = pd.concat([df_final, full_df[['event_id', 'data']]], axis=1)

In [30]:
result = result.loc[:, ~result.columns.duplicated()]

In [31]:
all_bid_lot['Savings'] = all_bid_lot['Savings'].astype(float)

In [32]:
# Grouping and counting as per the original logic
all_bid_lot_no = all_bid_lot.groupby(['Categ_L1','supplier'])['Lot/Item Bid Version'].count().reset_index()
all_bid_lot_mean = all_bid_lot_no.groupby('Categ_L1')['Lot/Item Bid Version'].mean().reset_index()
all_bid_lot_mean_supplier = all_bid_lot.groupby('Categ_L1')['supplier'].count().reset_index()
all_bid_lot_mean_item = all_bid_lot.groupby('Categ_L1')['Item Name'].count().reset_index()
all_bid_lot_mean_savings = all_bid_lot.groupby('Categ_L1')['Savings'].mean().reset_index()
accepted_bids = all_bid_lot[all_bid_lot['Bid Status'] == 'Accepted']
all_bid_lot_bid_status_count = accepted_bids.groupby('Categ_L1')['Bid Status'].count().reset_index()

# Merging the grouped data into one DataFrame
counts_all_bid = all_bid_lot_mean.rename(columns={'Lot/Item Bid Version': 'Mean_Lot_Item_Bid_Version'}) \
    .merge(all_bid_lot_mean_supplier.rename(columns={'supplier': 'Count_supplier'}), on='Categ_L1') \
    .merge(all_bid_lot_mean_item.rename(columns={'Item Name': 'Count_Item_Name'}), on='Categ_L1') \
    .merge(all_bid_lot_mean_savings.rename(columns={'Savings': 'Average_savings'}), on='Categ_L1') \
    .merge(all_bid_lot_bid_status_count.rename(columns={'Bid Status': 'Count_Accepted'}), on='Categ_L1')\
    .merge(all_bid_lot_no.groupby('Categ_L1')['Lot/Item Bid Version'].count().reset_index().rename(columns={'Lot/Item Bid Version': 'Count_Lot_Item_Bid_Version'}), on='Categ_L1')




In [33]:
merged_df = pd.merge(full_df, counts_all_bid, on='Categ_L1', how='left')

In [34]:
result1=pd.merge(result, merged_df[['event_id','Mean_Lot_Item_Bid_Version','Count_supplier','Count_Item_Name','Count_Lot_Item_Bid_Version','Count_Accepted','Average_savings']], on=['event_id'], how='left')

In [35]:
result1

,baselinespend_usd,bestbid_usd,finalspend_usd,effectiveness_usd,efficiency,participant,itemno,ratio,bidrankthattriggersovertime,biddingperiod,...,setareviewperiodafterlotcloses_Yes,day_type_Weekend,event_id,data,Mean_Lot_Item_Bid_Version,Count_supplier,Count_Item_Name,Count_Lot_Item_Bid_Version,Count_Accepted,Average_savings
0,123216.00,162000.63,111294.73,50705.90,0.312998,3.0,1.0,3.0,3.0,20.0,...,1,0,FULA,train,262.465034,739364,739364,2817,54418,-2.654656e+05
1,217168.38,217168.38,212932.80,4235.58,0.019504,4.0,1.0,4.0,0.0,30.0,...,1,0,GAEQ,train,77.891784,349812,349812,4491,30147,4.175511e+07
2,5744.16,5744.16,5744.16,0.00,0.000000,1.0,1.0,1.0,1.0,0.0,...,1,0,AJUL,train,77.891784,349812,349812,4491,30147,4.175511e+07
3,61179.86,61179.86,61179.86,0.00,0.000000,6.0,1.0,6.0,0.0,30.0,...,1,0,CJFZ,train,77.891784,349812,349812,4491,30147,4.175511e+07
4,126436.10,126436.10,121416.58,5019.52,0.039700,4.0,1.0,4.0,0.0,30.0,...,1,0,BMRW,train,77.891784,349812,349812,4491,30147,4.175511e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19708,500489.70,295250.50,0.00,0.00,0.000000,22.0,2.0,11.0,1.0,0.0,...,0,0,AIAA,submission_df,135.246032,187451,187451,1386,11701,1.517529e+12
19709,36523.77,36523.77,0.00,0.00,0.000000,4.0,2.0,2.0,2.0,45.0,...,0,0,DYSS,submission_df,77.891784,349812,349812,4491,30147,4.175511e+07
19710,3442025.08,3442025.08,0.00,0.00,0.000000,12.0,3.0,4.0,2.0,30.0,...,1,0,UGX,submission_df,77.891784,349812,349812,4491,30147,4.175511e+07
19711,116631.46,116631.46,0.00,0.00,0.000000,5.0,1.0,5.0,1.0,30.0,...,1,1,EEEM,submission_df,77.891784,349812,349812,4491,30147,4.175511e+07


In [37]:
result1.to_csv(r'Data\NEW_ADS3.csv')